In [1]:
import pandas as pd
from tqdm.notebook import tqdm, tnrange

user_first_deposits_df = pd.read_json('user_first_deposits_part_14.json', orient='records', lines=True, dtype={'first_deposit_date': 'datetime64[ns]'}, convert_dates=['first_deposit_date'])

max_date_df = pd.read_gbq(query='(select max(date(deposit_initiated_ts)) from `transactions.fact_deposits`)')
max_date = max_date_df.iloc[0,0]

_user_table_script_query = None

with open('user_table_sript.sql', 'r') as f:
    _user_table_script_query = f.read()
    
def collect_and_write_user_table(user_address, first_deposit_date, max_date):
    from google.cloud import bigquery

    # Construct a BigQuery client object.
    client = bigquery.Client()

    # Run a SQL script.
    sql_script = _user_table_script_query
    
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("user_address", "STRING", user_address),
            bigquery.ScalarQueryParameter("first_deposit_dt", "DATE", first_deposit_date),
            bigquery.ScalarQueryParameter("max_date", "DATE", max_date)
        ]
    )

    parent_job = client.query(sql_script, job_config=job_config)

    # Wait for the whole script to finish.
    rows_iterable = parent_job.result()
    # print("Script created {} child jobs.".format(parent_job.num_child_jobs))

    # Fetch result rows for the final sub-job in the script.
    rows = list(rows_iterable)
    
    print_true = (len(rows) > abs((user_first_deposits_df.first_deposit_date.min().date() - max_date).days)+1)
    
    if print_true:
        print(
            f'{len(rows)} rows of user data were collected for user {user_address}'
            )


    # Fetch jobs created by the SQL script.
    child_jobs_iterable = client.list_jobs(parent_job=parent_job)
    for child_job in child_jobs_iterable:
        child_rows = list(child_job.result())
        # print(
        #     "Child job with ID {} produced {} row(s).".format(
        #         child_job.job_id, len(child_rows)
        #     )
        # )
        
    user_df = None
    for job in client.list_jobs(parent_job=parent_job):
        user_df = client.get_job(job.job_id).to_dataframe().sort_values('as_of_date')
        break
    
    user_df.to_json(f'gcs://friktion-users-prod/user-{user_address}.json', orient='records', date_format='iso', lines=True, date_unit='s')
    if print_true:
        print(f'Written to gcs://friktion-users-prod/user-{user_address}.json')
        
for row in tqdm(user_first_deposits_df.itertuples()):
    collect_and_write_user_table(row.user_address, row.first_deposit_date.date(), max_date)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  after removing the cwd from sys.path.


0it [00:00, ?it/s]